# 📊 Dataset Sintético SBA México - Análisis y Uso

Este notebook demuestra cómo usar el dataset sintético de créditos PyME México,
equivalente al dataset SBA de Estados Unidos.

**Contenido:**
1. Carga y exploración del dataset
2. Mapeo de variables SBA → México
3. Análisis exploratorio
4. Ejemplo de modelo de predicción de default
5. Cómo adaptar tu modelo existente

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

## 1. Carga del Dataset

In [ ]:
# Cargar dataset
df = pd.read_csv('sba_mexico_sintetico.csv', parse_dates=['ApprovalDate', 'DisbursementDate', 'ChgOffDate'])

print(f"Dimensiones del dataset: {df.shape}")
print(f"\nColumnas disponibles:")
print(df.columns.tolist())

In [ ]:
# Vista general
df.head()

In [ ]:
# Estadísticas descriptivas
df.describe()

## 2. Mapeo de Variables SBA ↔ México

| Variable SBA Original | Variable México | Notas |
|----------------------|-----------------|-------|
| NAICS | SCIAN | Compatible a 2 dígitos |
| State | State | Códigos de estados mexicanos |
| SBA_Appv | NAFIN_Appv | Garantía de Nacional Financiera |
| LowDoc | - | No hay programa equivalente directo |
| Default | Default | Misma definición |
| ChgOffPrinGr | ChgOffPrinGr | En MXN en lugar de USD |

In [ ]:
# Diccionario de sectores SCIAN
SECTORES_SCIAN = {
    '11': 'Agricultura, ganadería, pesca',
    '21': 'Minería',
    '22': 'Generación de energía',
    '23': 'Construcción',
    '31': 'Manufactura - Alimentos',
    '32': 'Manufactura - Textil/Química',
    '33': 'Manufactura - Metálica/Maquinaria',
    '43': 'Comercio al por mayor',
    '46': 'Comercio al por menor',
    '48': 'Transporte',
    '51': 'Información en medios',
    '52': 'Servicios financieros',
    '53': 'Servicios inmobiliarios',
    '54': 'Servicios profesionales',
    '56': 'Servicios de apoyo',
    '61': 'Servicios educativos',
    '62': 'Servicios de salud',
    '71': 'Esparcimiento/Cultura',
    '72': 'Alojamiento/Alimentos',
    '81': 'Otros servicios',
}

df['Sector_Nombre'] = df['SCIAN'].map(SECTORES_SCIAN)

## 3. Análisis Exploratorio

In [ ]:
# Distribución de la variable objetivo
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Default rate
default_counts = df['Default'].value_counts()
axes[0].pie(default_counts, labels=['No Default', 'Default'], autopct='%1.1f%%', 
            colors=['#2ecc71', '#e74c3c'], explode=[0, 0.05])
axes[0].set_title('Distribución de Defaults')

# Default rate por año
default_by_year = df.groupby('ApprovalFY')['Default'].mean() * 100
axes[1].bar(default_by_year.index, default_by_year.values, color='steelblue')
axes[1].axhline(y=df['Default'].mean()*100, color='red', linestyle='--', label='Promedio')
axes[1].set_xlabel('Año de Aprobación')
axes[1].set_ylabel('Tasa de Default (%)')
axes[1].set_title('Tasa de Default por Año')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Default por sector
fig, ax = plt.subplots(figsize=(12, 6))

default_by_sector = df.groupby('Sector_Nombre').agg({
    'Default': 'mean',
    'LoanNr_ChkDgt': 'count'
}).sort_values('Default', ascending=True)

colors = ['#e74c3c' if x > 0.10 else '#f39c12' if x > 0.07 else '#2ecc71' 
          for x in default_by_sector['Default']]

bars = ax.barh(default_by_sector.index, default_by_sector['Default'] * 100, color=colors)
ax.axvline(x=df['Default'].mean()*100, color='black', linestyle='--', linewidth=2, label='Promedio')
ax.set_xlabel('Tasa de Default (%)')
ax.set_title('Tasa de Default por Sector SCIAN')
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Distribución de montos
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma de montos (log scale)
axes[0].hist(df['GrAppv']/1e6, bins=50, color='steelblue', edgecolor='white', alpha=0.7)
axes[0].set_xlabel('Monto Aprobado (Millones MXN)')
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribución de Montos Aprobados')
axes[0].set_xlim(0, 5)  # Limitar a 5 millones para mejor visualización

# Boxplot por tamaño de empresa
df['Tamano'] = pd.cut(df['NoEmp'], bins=[0, 10, 50, 250, 1000], 
                      labels=['Micro', 'Pequeña', 'Mediana', 'Grande'])

df.boxplot(column='GrAppv', by='Tamano', ax=axes[1])
axes[1].set_ylabel('Monto Aprobado (MXN)')
axes[1].set_title('Monto por Tamaño de Empresa')
axes[1].ticklabel_format(style='plain', axis='y')
plt.suptitle('')

plt.tight_layout()
plt.show()

In [ ]:
# Mapa de calor: Default por Estado
default_by_state = df.groupby('State')['Default'].agg(['mean', 'count'])
default_by_state.columns = ['Tasa_Default', 'Num_Prestamos']
default_by_state = default_by_state.sort_values('Num_Prestamos', ascending=False).head(15)

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(default_by_state.index, default_by_state['Tasa_Default'] * 100, 
              color='steelblue', edgecolor='white')
ax.axhline(y=df['Default'].mean()*100, color='red', linestyle='--', label='Promedio Nacional')
ax.set_xlabel('Estado')
ax.set_ylabel('Tasa de Default (%)')
ax.set_title('Tasa de Default por Estado (Top 15 por volumen)')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. Preparación de Datos para Modelado

In [ ]:
# Seleccionar features para el modelo (similar a tu modelo SBA)
features = [
    'GrAppv',           # Monto aprobado
    'NAFIN_Appv',       # Monto garantizado (equivalente a SBA_Appv)
    'Term',             # Plazo
    'NoEmp',            # Número de empleados
    'NewExist',         # Nuevo/Existente
    'UrbanRural',       # Urbano/Rural
    'RealEstate',       # Garantía inmobiliaria
    'Portion',          # Proporción garantizada
    'Recession',        # Indicador de recesión
    'New',              # Variable binaria nuevo
]

# Variables categóricas a codificar
categorical_features = ['State', 'SCIAN', 'Bank']

# Crear copia para modelado
df_model = df.copy()

In [ ]:
# Codificar variables categóricas
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    df_model[col + '_encoded'] = le.fit_transform(df_model[col].astype(str))
    label_encoders[col] = le
    features.append(col + '_encoded')

# Convertir RevLineCr a numérico
df_model['RevLineCr_num'] = (df_model['RevLineCr'] == 'Y').astype(int)
features.append('RevLineCr_num')

print(f"Features seleccionados: {len(features)}")
print(features)

In [ ]:
# Preparar X e y
X = df_model[features].copy()
y = df_model['Default'].copy()

# Verificar valores faltantes
print("Valores faltantes por columna:")
print(X.isnull().sum())

# Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTrain set: {len(X_train):,} registros")
print(f"Test set: {len(X_test):,} registros")
print(f"\nTasa de default en train: {y_train.mean()*100:.2f}%")
print(f"Tasa de default en test: {y_test.mean()*100:.2f}%")

## 5. Entrenamiento de Modelo de Ejemplo

In [ ]:
# Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=20,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

# Predicciones
y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]

print("Random Forest - Resultados en Test Set")
print("="*50)
print(classification_report(y_test, y_pred_rf))
print(f"\nAUC-ROC: {roc_auc_score(y_test, y_prob_rf):.4f}")

In [ ]:
# Gradient Boosting (alternativa)
gb_model = GradientBoostingClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)

gb_model.fit(X_train, y_train)

y_pred_gb = gb_model.predict(X_test)
y_prob_gb = gb_model.predict_proba(X_test)[:, 1]

print("Gradient Boosting - Resultados en Test Set")
print("="*50)
print(classification_report(y_test, y_pred_gb))
print(f"\nAUC-ROC: {roc_auc_score(y_test, y_prob_gb):.4f}")

In [ ]:
# Comparar curvas ROC
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Curvas ROC
for name, y_prob, color in [('Random Forest', y_prob_rf, 'blue'), 
                             ('Gradient Boosting', y_prob_gb, 'green')]:
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    auc = roc_auc_score(y_test, y_prob)
    axes[0].plot(fpr, tpr, color=color, label=f'{name} (AUC = {auc:.4f})')

axes[0].plot([0, 1], [0, 1], 'k--', label='Random')
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('Curvas ROC')
axes[0].legend()

# Feature importance
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=True)

axes[1].barh(importance_df['Feature'], importance_df['Importance'], color='steelblue')
axes[1].set_xlabel('Importancia')
axes[1].set_title('Importancia de Variables (Random Forest)')

plt.tight_layout()
plt.show()

## 6. Predicción de Pérdida (ChgOffPrinGr)

Para predecir la pérdida monetaria, usamos un modelo de dos etapas:
1. Predecir probabilidad de default
2. Predecir pérdida condicional al default

In [ ]:
# Para los defaults, entrenar modelo de regresión para ChgOffPrinGr
from sklearn.ensemble import RandomForestRegressor

# Filtrar solo defaults
df_defaults = df_model[df_model['Default'] == 1].copy()
print(f"Registros con default: {len(df_defaults):,}")

# Features para regresión
X_loss = df_defaults[features]
y_loss = df_defaults['ChgOffPrinGr']

# Train/test split
X_train_loss, X_test_loss, y_train_loss, y_test_loss = train_test_split(
    X_loss, y_loss, test_size=0.2, random_state=42
)

# Entrenar modelo de regresión
loss_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

loss_model.fit(X_train_loss, y_train_loss)

# Evaluar
y_pred_loss = loss_model.predict(X_test_loss)

from sklearn.metrics import mean_absolute_error, r2_score

print(f"MAE: ${mean_absolute_error(y_test_loss, y_pred_loss):,.2f} MXN")
print(f"R²: {r2_score(y_test_loss, y_pred_loss):.4f}")

In [ ]:
# Visualizar predicciones vs reales
fig, ax = plt.subplots(figsize=(8, 8))

ax.scatter(y_test_loss/1e6, y_pred_loss/1e6, alpha=0.3, color='steelblue')
ax.plot([0, y_test_loss.max()/1e6], [0, y_test_loss.max()/1e6], 'r--', label='Perfecta')
ax.set_xlabel('Pérdida Real (Millones MXN)')
ax.set_ylabel('Pérdida Predicha (Millones MXN)')
ax.set_title('Predicción de Pérdida (ChgOffPrinGr)')
ax.legend()
ax.set_xlim(0, 2)
ax.set_ylim(0, 2)

plt.tight_layout()
plt.show()

## 7. Cómo Adaptar tu Modelo SBA Existente

Si ya tienes un modelo entrenado con datos SBA de EE.UU., puedes adaptarlo así:

In [ ]:
# Función para renombrar columnas de México a formato SBA
def convertir_a_formato_sba(df_mexico):
    """
    Convierte el dataset México al formato exacto del dataset SBA
    para compatibilidad con modelos existentes.
    """
    df_sba = df_mexico.copy()
    
    # Renombrar columnas principales
    renombres = {
        'SCIAN': 'NAICS',           # Sistema de clasificación compatible
        'NAFIN_Appv': 'SBA_Appv',   # Garantía gubernamental
    }
    df_sba = df_sba.rename(columns=renombres)
    
    # Agregar columna LowDoc (no existe equivalente, ponemos N)
    df_sba['LowDoc'] = 'N'
    
    return df_sba

# Ejemplo de uso
df_sba_format = convertir_a_formato_sba(df)
print("Columnas en formato SBA:")
print([c for c in df_sba_format.columns if c in ['NAICS', 'SBA_Appv', 'LowDoc', 'GrAppv', 'Default']])

In [ ]:
# Función para transfer learning (fine-tuning)
def fine_tune_modelo_sba(modelo_sba_original, X_mexico, y_mexico, epochs=5):
    """
    Ejemplo conceptual de cómo hacer fine-tuning de un modelo SBA
    con datos mexicanos.
    
    Para modelos scikit-learn: usar warm_start=True
    Para redes neuronales: entrenar capas finales con learning rate bajo
    """
    # Para Random Forest con warm_start
    modelo_mexico = modelo_sba_original
    modelo_mexico.n_estimators += 50  # Agregar más árboles
    modelo_mexico.fit(X_mexico, y_mexico)
    
    return modelo_mexico

print("Para hacer transfer learning:")
print("1. Carga tu modelo SBA entrenado")
print("2. Convierte datos México al formato SBA")
print("3. Haz fine-tuning con los datos mexicanos")
print("4. Evalúa en datos de prueba mexicanos")

## 8. Exportar Modelo para Producción

In [ ]:
import joblib

# Guardar modelos
joblib.dump(rf_model, 'modelo_default_rf_mexico.pkl')
joblib.dump(loss_model, 'modelo_perdida_mexico.pkl')
joblib.dump(label_encoders, 'label_encoders_mexico.pkl')

print("Modelos guardados:")
print("  - modelo_default_rf_mexico.pkl")
print("  - modelo_perdida_mexico.pkl")
print("  - label_encoders_mexico.pkl")

In [ ]:
# Función de predicción completa
def predecir_riesgo_credito(datos_prestamo, modelo_default, modelo_perdida, encoders):
    """
    Predice probabilidad de default y pérdida esperada.
    
    Parámetros:
    -----------
    datos_prestamo : dict
        Diccionario con características del préstamo
    
    Retorna:
    --------
    dict con prob_default, perdida_esperada, recomendacion
    """
    # Preparar datos
    df_input = pd.DataFrame([datos_prestamo])
    
    # Codificar categóricas
    for col, encoder in encoders.items():
        if col in df_input.columns:
            df_input[col + '_encoded'] = encoder.transform(df_input[col].astype(str))
    
    # Predecir default
    prob_default = modelo_default.predict_proba(df_input[features])[:, 1][0]
    
    # Predecir pérdida condicional
    perdida_condicional = modelo_perdida.predict(df_input[features])[0]
    
    # Pérdida esperada = P(default) * Pérdida|default
    perdida_esperada = prob_default * perdida_condicional
    
    # Recomendación
    if prob_default < 0.05:
        recomendacion = 'APROBAR - Riesgo bajo'
    elif prob_default < 0.15:
        recomendacion = 'REVISAR - Riesgo moderado'
    else:
        recomendacion = 'RECHAZAR - Riesgo alto'
    
    return {
        'probabilidad_default': prob_default,
        'perdida_condicional': perdida_condicional,
        'perdida_esperada': perdida_esperada,
        'recomendacion': recomendacion
    }

# Ejemplo de uso
ejemplo_prestamo = {
    'GrAppv': 500000,
    'NAFIN_Appv': 300000,
    'Term': 36,
    'NoEmp': 5,
    'NewExist': 2,
    'UrbanRural': 1,
    'RealEstate': 0,
    'Portion': 0.6,
    'Recession': 0,
    'New': 0,
    'State': 'JAL',
    'SCIAN': '46',
    'Bank': 'BBVA México',
    'RevLineCr': 'N'
}

print("\nEjemplo de predicción:")
print(f"Préstamo: ${ejemplo_prestamo['GrAppv']:,} MXN")
# resultado = predecir_riesgo_credito(ejemplo_prestamo, rf_model, loss_model, label_encoders)
# print(resultado)

## 9. Resumen y Próximos Pasos

### Dataset Generado:
- **50,000 registros** de préstamos PyME sintéticos
- **32 variables** compatibles con estructura SBA
- **Calibrado** con datos reales de CNBV, INEGI y Banxico
- **Tasa de default realista**: ~10% (dentro del rango histórico mexicano)

### Cómo mejorar el dataset:

1. **Agregar datos reales**: Si consigues acceso a microdatos de ENAFIN a través de solicitud formal a INEGI

2. **Calibrar tasas de default por sector**: Usar reportes trimestrales de CNBV sobre IMOR

3. **Incluir variables macroeconómicas**: Tasa de interés Banxico, inflación, tipo de cambio

4. **Expandir a LATAM**: Agregar datos de Colombia (SFC), Brasil (BCB), Chile (CMF)

### Fuentes de datos adicionales:
- CNBV Portafolio de información: https://portafoliodeinformacion.cnbv.gob.mx/
- Banxico Sistema de Información Económica: https://www.banxico.org.mx/SieInternet/
- INEGI Censos Económicos: https://www.inegi.org.mx/programas/ce/2024/